In [1]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from seleniumwire import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [2]:
proxy_address = ""  # Replace with your proxy details!
proxy_port = ""
proxy_username = "" # Replace if your proxy needs authentication
proxy_password = "" # Replace if your proxy needs authentication

In [3]:
# proxy_url = f"http://{proxy_username}:{proxy_password}@{proxy_address}:{proxy_port}"

# # set selenium-wire options to use the proxy
# seleniumwire_options = {
#     "proxy": {
#         "http": proxy_url,
#         "https": proxy_url
#     },
# }

# set Chrome options to run in headless mode
# options = Options()
# options.add_argument("--headless=new")

# initialize the Chrome driver with service, selenium-wire options, and chrome options
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    # seleniumwire_options=seleniumwire_options
    # options=options
)

# navigate to the target webpage
driver.get("https://www.tradeindia.com/manufacturers/a-alphabet/")

In [4]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')
url_list=[]
profiles = soup.find_all('a', {'class': "product-Link-cont active pageflex-A"})
for profile in profiles:
    url_list.append(profile['href'])

In [5]:
listings = []
for url in url_list:
    driver.get(url)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    company_listing = soup.find_all('a', {'class': "company-url"})
    for list in company_listing:
        listings.append(list['href'])

In [6]:
print(len(listings))
driver.quit()

2643


In [7]:
# proxy_url = f"http://{proxy_username}:{proxy_password}@{proxy_address}:{proxy_port}"

# # set selenium-wire options to use the proxy
# seleniumwire_options = {
#     "proxy": {
#         "http": proxy_url,
#         "https": proxy_url
#     },
# }

# set Chrome options to run in headless mode
# options = Options()
# options.add_argument("--headless=new")

# initialize the Chrome driver with service, selenium-wire options, and chrome options
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    # seleniumwire_options=seleniumwire_options
    # options=options
)


In [8]:
manufacturer_list = []
for i in range (len(listings)):
    list = {}
    try:
        driver.get(listings[i])
        page_source = driver.page_source
        soup = BeautifulSoup(page_source,'html.parser')
        driver.set_page_load_timeout(30)

        try:
            name = soup.find('h1',{'class':'sc-3b1eb120-11 RjdPL mb-1 Body2R'}).text.strip()
        except:
            url = listings[i]
            manufacturer_list['url'] = url
            manufacturer_list.append("This url can't be scrapped")
            continue

        url = listings[i]

        about = soup.find('div',{'class':'sc-3b1eb120-11 iUzTdD full-details Body2R pb-1'}).text.strip()

        try:
            number = soup.find('a',{'class':'sc-c9bc48ef-0 fAwUDn'})['href'].split(':')[1].strip()
        except:
            number = 'NA'

        try:
            gst_no = soup.find('span',{'class':'no-wrap'}).text.split(' ')[1]
        except:
            gst_no = 'NA'

        try:
            rating = soup.find('span',{'class':'mr-1'}).text.strip()
        except:
            rating = 'NA'

        head= soup.find('p',{'class':'sc-3b1eb120-13 jSQhul m-0 mb-3 Body4R'}).text.strip()

        common = soup.find_all('p',{'class':'sc-3b1eb120-13 jSQhul'})
        if len(common)>2:
            member_since = common[1]
            member_since = member_since.text.strip()
            address = common[2]
            address = address.text.strip()
        else:
            member_since = 'NA'
            try:
                address = common[1]
                address = address.text.strip()
            except:
                address = 'NA'
            
        title = soup.find_all('h3', {'class':'sc-3b1eb120-8 gWoKIS mt-2 mb-1 BodyTitle2S'})
        content = soup.find_all('p', {'class':'sc-3b1eb120-13 jSQhul Body3R'})
        info = {}
        for items in range(len(title)):
            info[title[items].text.strip()]=content[items].text.strip()

        list['name'] = name
        list['url'] = url
        list['gst_no'] = gst_no
        list['head'] = head
        list['about'] = about
        # list['number'] = number
        list['rating'] = rating
        list['member_since'] = member_since
        list['address'] = address
        list['info'] = info
        manufacturer_list.append(list)

    except:
        continue

In [ ]:
driver.quit()

In [9]:
len(manufacturer_list)

2566

In [10]:
import json

with open(f'manufacturer_list.json', 'w') as f:
    json.dump(manufacturer_list, f, indent=4)

In [21]:
import pandas as pd
data = pd.read_csv('D:/trade_India_scraper/manufacturer_list.csv')
data.head()

ParserError: Error tokenizing data. C error: Expected 30 fields in line 3, saw 32
